In [45]:
import geopandas as gpd
import os
import pandas as pd
import openrouteservice
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import contextily as ctx
os.chdir("/home/silas/projects/msc_thesis")

figure_dir = "docs/reports/thesis/figures/"

In [38]:
# import flats dataset with duration to nearest recycling point
flats_duration = gpd.read_file(r'data/derived_data/flats_subset_with_rcp.shp')

#import rcp dataset
rcps = gpd.read_file(r'./data/raw_data/geodata_stadt_Zuerich/recycling_sammelstellen/data/stzh.poi_sammelstelle_view.shp')

# convert crs to 3857
flats_duration = flats_duration.to_crs(epsg=4326)
rcps = rcps.to_crs(epsg=4326)


In [48]:

# load the data


# visualize the results 
# Create a base map
m = folium.Map(location=[47.3769, 8.5417], zoom_start=12)

# Create a colormap for the duration
colormap = folium.LinearColormap(colors=['green', 'yellow', 'red'], vmin=flats_duration['duration'].min(), vmax=flats_duration['duration'].max(), caption='Duration to Closest RCP (minutes)')

# Add flats to the map
for idx, row in flats_duration.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=6,
        color=colormap(row['duration']),
        fill=True,
        fill_color=colormap(row['duration']),
        fill_opacity=0.7,
        popup=f"Flat ID: {row['egid']}<br>Duration: {row['duration']:.2f} min"
    ).add_to(m)

# Add recycling points to the map
for idx, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        icon=folium.Icon(color='green', icon='recycle', prefix='fa', ),
        popup=f"RCP ID: {idx}<br>Address: {row['adresse']}", radius=3
    ).add_to(m)

# Add the colormap to the map
colormap.add_to(m)

m.save(figure_dir + 'points_rcp_map.html')
# Display the map
#m

In [49]:

# create a heat map with folium

# Create a base map
m = folium.Map(location=[47.3769, 8.5417], zoom_start=12)

# Prepare data for heatmap
heat_data = [[row.geometry.y, row.geometry.x, row['duration']] for idx, row in flats_subset_with_rcp.iterrows()]

# Add recycling points to the map
for idx, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        fill=True,
        icon=folium.Icon(color='green', icon='recycle', prefix='fa'),
        fill_opacity=0.7,
        color='red',
        popup=f"RCP ID: {idx}<br>Address: {row['adresse']}"
    ).add_to(m)

# Create and add heatmap to the map
HeatMap(heat_data, radius=15, blur=12, max_zoom=1).add_to(m)

# save the map
m.save('./data/derived_data/map_flats_rcps_heatmap.html')
m


